# Tests for Amphibole functions

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("example_table.csv")

In [3]:
df['Fe2O3'] = 0

In [4]:
from Amphibole import normalization

In [5]:
normalized_dataset = normalization(df)

In [6]:
# dataset with elements normalized, must add some other calculations
normalized_dataset

,Si,Ti,Al,Fe2+,Fe3+,Mn,Mg,Ca,Na,K
0,6.271672,0.138655,2.311706,0.272426,0.886155,0.000000,3.119387,1.581527,0.700852,0.117580
1,7.663776,0.000000,0.757001,2.490403,0.000000,0.037093,3.917099,0.078201,0.028304,0.000000
2,6.651607,0.020417,2.384953,0.062425,0.000000,0.000000,5.790553,0.033445,0.384196,0.000000
3,6.070040,0.000000,3.208801,2.920900,0.189580,0.050405,2.528394,0.031880,0.461540,0.000000
4,6.080335,0.057155,3.679175,4.703644,0.000000,0.308388,0.007080,0.005088,0.356040,0.008078
5,7.938361,0.000000,0.158186,1.556215,0.000000,0.039078,5.150113,0.124329,0.032529,0.000000
6,7.839638,0.000000,0.281590,3.399041,0.010181,0.101184,3.272371,0.095994,0.028953,0.000000
7,7.998719,0.000000,0.022831,5.418888,0.000000,0.251582,1.176777,0.121069,0.000000,0.000000
8,7.929442,0.000000,0.007388,4.612489,0.118537,0.769849,1.495136,0.067160,0.015192,0.000000
9,7.989093,0.000000,0.057922,0.546390,0.000000,2.034814,4.132998,0.184295,0.072868,0.000000
